<a href="https://colab.research.google.com/github/donghuna/PromptGenerate/blob/main/generateCode-codeLlama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install peft
!pip install datasets
# !pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datetime import datetime
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
# from peft import prepare_model_for_kbit_training, LoraConfig, PeftModel, get_peft_model
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import csv

In [3]:
from google.colab import userdata

from huggingface_hub import login
login(token=userdata.get('HUG_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')


In [5]:

tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")
pipeline = transformers.pipeline(
    "text-generation",
    model="codellama/CodeLlama-7b-hf",
    torch_dtype=torch.float16,
    device_map="auto",
)


# ### Load Model Normaly ###

# model_path = "codellama/CodeLlama-7b-Instruct-hf"

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# tokenizer = AutoTokenizer.from_pretrained(
#     model_path,
#     # padding_side="left",
#     add_eos_token=True)

# model = AutoModelForCausalLM.from_pretrained(
#     model_path,
#     torch_dtype=torch.float16,
#     pad_token_id=tokenizer.eos_token_id,
#     device_map="auto")

# # pad_token_id 설정
# # model.config.pad_token_id = model.config.eos_token_id


# # # Load Tokenizer
# # tokenizer = AutoTokenizer.from_pretrained(
# #     model_path,
# #     model_max_length=512,
# #     padding_side="left",
# #     add_eos_token=True,
# # )
# # tokenizer.pad_token = tokenizer.eos_token

# # # Load Model without quantization
# # model = AutoModelForCausalLM.from_pretrained(
# #     model_path,
# #     torch_dtype=torch.float16,  # GPU 메모리 절약을 위해 float16 사용 (필요시 float32로 변경 가능)
# #     device_map="auto",  # 모델을 자동으로 GPU로 로드
# # )

# model.eval()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
dataset = load_dataset("openai_humaneval")
dataset

README.md:   0%|          | 0.00/6.52k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
        num_rows: 164
    })
})

In [7]:
for idx, data in enumerate(dataset["test"]):
        task_id = data["task_id"]
        data_prompt = data["prompt"]

        sequences = pipeline(
            data_prompt,
            do_sample=True,
            temperature=0.2,
            top_p=0.9,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.2,
            max_length=500,
        )
        print(sequences[0]['generated_text'])
        break


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    for i in range(len(numbers)):
        for j in range(i + 1, len(numbers)):
            diff = abs(numbers[j] - numbers[i])
            if diff < threshold:
                return True
    return False



In [8]:



# 3. Text generation 파이프라인 생성
# generate_code = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)


# 4. CSV 파일 생성 및 작성 준비
output_file = "generated_code-onlyPrompt.csv"
with open(output_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["task_id", "Prompt", "Generated Code"])  # CSV 헤더 작성

    # 5. HumanEval 데이터셋의 각 prompt를 모델에 입력하여 코드 생성 및 저장
    for idx, data in enumerate(dataset["test"]):
        task_id = data["task_id"]
        system_prompt = "You are an AI specialized in generating Python code. Only produce Python code that directly corresponds to the provided comments or function descriptions without any additional explanations or extraneous text."
        data_prompt = data["prompt"]
        prompt = f"""<s>[INST] <<SYS>>\\n{system_prompt}\\n<</SYS>>\\n\\n{data_prompt} [/INST]"""

        # 입력 토큰 길이 계산
        encoding = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
        input_ids = encoding.input_ids
        # input_length = input_ids.shape[1]



        # CodeLlama 모델로 코드 생성
        output = model.generate(
            input_ids.to(model.device),
            max_new_tokens=200,
            num_return_sequences=1,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id,
            attention_mask=encoding.attention_mask)

        output_text = tokenizer.decode(output[0], skip_special_tokens=True)
        # output_text = tokenizer.decode(output[0][input_length:], skip_special_tokens=True)
        generated_code = output_text[len(prompt):].strip()
        print(output_text)
        # print(input_length)
        # print(generated_code)


        # generated_code = generate_code(prompt, max_new_tokens=100, num_return_sequences=1, do_sample=True)[0]["generated_text"]

        # 입력 프롬프트 이후에 생성된 토큰만 추출
        # generated_code_only = generated_code[len(prompt):].strip()

        # # CSV 파일에 prompt와 generated_code 추가
        # writer.writerow([task_id, data_prompt, generated_code_only.replace('\n', '\\n')])
        break

        # print(f"Saved Prompt {idx+1} to CSV")

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
# !cp generated_code-onlyPrompt.csv /content/drive/MyDrive/
